In [2]:
import pandas as pd
URLCSV = "https://docs.google.com/spreadsheets/d/1Fuj72q5oIxLK74jBfdvI9pNr5Er2rPONfwh3szMq6-Q/export?format=csv"
dane = pd.read_csv(URLCSV)
print(dane.head())

   height  handedness  right_hand_span  left_hand_span  head_circ eye_colour  \
0     159        0.88             19.0            19.0       54.0       Blue   
1     160       -1.00             19.0            20.0       57.0      Green   
2     161        0.79             17.0            16.5       57.0      hazel   
3     161        1.00             17.0            17.5       54.0       Blue   
4     162        0.79             16.0            16.0       57.0       gray   

   gender  siblings  movies  soda   bedtime       fb_freq  fb_friends  \
0  Female         2     3.0   7.0  02:00:00    once a day       135.0   
1  Female         2     0.5   2.0  04:30:00             0         1.0   
2  Female         3     3.0   2.0  23:50:00   once a week       354.0   
3  Female         2     2.0   0.5  23:00:00    once a day       200.0   
4  Female         2     0.0   2.0  23:10:00  almost never       192.0   

                  stat_likert  
0  Neither agree nor disagree  
1              S

In [3]:
import numpy as np
np.mean(dane['head_circ'])

np.float64(57.011842105263156)

In [5]:
import math
import scipy.stats as stats

data = dane['head_circ'].dropna()
test_result = stats.ttest_1samp(data, popmean=np.mean(data))
conf_int = test_result.confidence_interval(0.95)
print(conf_int)

ConfidenceInterval(low=np.float64(56.3453246623508), high=np.float64(57.678359548175514))


In [8]:
data = dane['movies'].dropna()
test_result = stats.ttest_1samp(data, popmean=np.mean(data))
conf_int = test_result.confidence_interval(0.95)
print(np.mean(data))
print(conf_int)

1.3982666666666668
ConfidenceInterval(low=np.float64(1.1304273840308394), high=np.float64(1.6661059493024941))


In [10]:
mask = dane['height'].notna() & dane['head_circ'].notna()
x = dane['height'][mask]
y = dane['head_circ'][mask]
stats.pearsonr(x, y)

PearsonRResult(statistic=np.float64(0.588700183421883), pvalue=np.float64(2.2294040330721613e-08))